In [1]:
import numpy as np
import matplotlib as mpl
mpl.use('Qt5Agg')
import matplotlib.pyplot as plt
import open3d
from intrinsics import Intrinsics
from matplotlib.widgets import RectangleSelector
import glob
import math
from scipy.spatial import Delaunay
from functools import reduce

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
len(glob.glob("depth/*.png"))

2786

In [4]:
def scale(depth_image, range):
    mask = (depth_image > range[0]) & (depth_image < range[1])
    depth_in_range = np.zeros(depth_image.shape)
    depth_in_range[mask] = depth_image[mask]

    return (depth_in_range - range[0]) / (range[1] - range[0])

In [5]:
range=(0, 3)
# depth_scale = 0.00025
# img_path = glob.glob("depth/*.png")[0]

depth = open3d.io.read_image("volume_tests/test_img_for_volume.png")
fig, ax = plt.subplots()
ax.imshow(depth, cmap="gray")

roi = None
def line_select_callback(eclick, erelease):
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata

    global roi
    roi = [[int(x1), int(y1)], [int(x2), int(y2)]]
    plt.close()

rs = RectangleSelector(ax, line_select_callback,
                       useblit=False, button=[1], 
                       minspanx=5, minspany=5, spancoords='pixels', 
                       interactive=True)

plt.connect('key_press_event', rs)
plt.show()

print(roi)

[[159, 135], [451, 476]]


In [6]:
npdepth = np.asanyarray(depth)
scaled = npdepth * 0.00025
cropped = scaled[roi[0][1]:roi[1][1], roi[0][0]:roi[1][0]]
o3d_cropped = open3d.geometry.Image(cropped.astype(np.float32))

In [7]:
intr = Intrinsics(width=640, height=480)

pcd = open3d.geometry.PointCloud.create_from_depth_image(o3d_cropped, intrinsic=intr.get_intrinsics())
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
axes = open3d.geometry.TriangleMesh.create_coordinate_frame()
open3d.visualization.draw_geometries([pcd, axes])

In [8]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.02,
                                         ransac_n=3,
                                         num_iterations=10000)
[a, b, c, d] = plane_model
plane_pcd = pcd.select_by_index(inliers)
plane_pcd.paint_uniform_color([1.0, 0, 0])
box_pcd = pcd.select_by_index(inliers, invert=True)
box_pcd.paint_uniform_color([0, 0, 1.0])
open3d.visualization.draw_geometries([plane_pcd, box_pcd, axes])

In [9]:
plane_model

array([-0.00231887,  0.69982064,  0.71431484,  0.60229289])

In [10]:
plane_pcd = plane_pcd.translate((0, d/b, 0))
box_pcd = box_pcd.translate((0, d/b, 0))

alpha = math.atan(a/b)
beta = math.atan(c/a)
gamma = -math.atan(c/b)
# 
R_z = np.array([
    [math.cos(alpha), -math.sin(alpha), 0],
    [math.sin(alpha), math.cos(alpha), 0],
    [0, 0, 1]
])

R_y = np.array([
    [math.cos(beta), 0, math.sin(beta)],
    [0, 1, 0],
    [-math.sin(beta), 0, math.cos(beta)]
])

R_x = np.array([
    [1, 0, 0],
    [0, math.cos(gamma), -math.sin(gamma)],
    [0, math.sin(gamma), math.cos(gamma)]
])
# 
R = R_z @ R_y @ R_x

rotated_plane_pcd = plane_pcd.rotate(R, center=(0, 0, 0))
rotated_box_pcd = box_pcd.rotate(R, center=(0, 0, 0))

open3d.visualization.draw_geometries([rotated_plane_pcd, rotated_box_pcd, axes])

In [11]:
cl, ind = rotated_box_pcd.remove_statistical_outlier(nb_neighbors=10,
                                            std_ratio=2.0)
rem_outlier_box_pcd = rotated_box_pcd.select_by_index(ind)
open3d.visualization.draw_geometries([rem_outlier_box_pcd])

In [14]:
downpdc = rem_outlier_box_pcd.voxel_down_sample(voxel_size=0.05)
xyz = np.asarray(downpdc.points)
xz_catalog = []
for point in xyz:
    xz_catalog.append(np.array([point[0], point[2]]))
    
xy_catalog = np.array(xz_catalog)
tri = Delaunay(xy_catalog)

In [16]:
open3d.visualization.draw_geometries([downpdc])

In [17]:
plt.triplot(xy_catalog[:,0], xy_catalog[:,1], tri.simplices)
plt.plot(xy_catalog[:,0], xy_catalog[:,1], 'o')
plt.show()

In [18]:
surface = open3d.geometry.TriangleMesh()
surface.vertices = open3d.utility.Vector3dVector(xyz)
surface.triangles = open3d.utility.Vector3iVector(np.flip(tri.simplices))
open3d.visualization.draw_geometries([surface], mesh_show_wireframe=True)

In [19]:
final_volume = 0

for triangle in tri.simplices:
    points = []
    for i in triangle:
        points.append(xyz[i])
        
    [p1, p2, p3] = points
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    x3, y3, z3 = p3
    
    base = .5*abs(x1*(z2-z3) + x2*(z3-z1) + x3*(y1 - y2))
    base_volume = base * np.min([y1, y2, y3])
    pyramid_volume = (1/3) * base * (np.max([y1, y2, y3]) - np.min([y1, y2, y3]))
    total_vol_under_triangle = base_volume + pyramid_volume
    final_volume += total_vol_under_triangle
    
final_volume
    

0.274489822256229